In [1]:
import os
import pandas as pd
import tensorflow
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import TextVectorization

In [2]:
train = pd.read_csv(r"..\dataset\jigsaw-toxic-comment-classification-challenge\train.csv\train.csv")
test = pd.read_csv(r"..\dataset\jigsaw-toxic-comment-classification-challenge\test.csv\test.csv")
test_lable= pd.read_csv(r"..\dataset\jigsaw-toxic-comment-classification-challenge\test_labels.csv\test_labels.csv")
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
x = train['comment_text']
y = train[train.columns[2:]].values
MAX_FEATURES = 200000  # number of words
vectorizer = TextVectorization(max_tokens = MAX_FEATURES,
                               output_mode = 'int',
                               output_sequence_length = 1800
                               )

In [6]:
vectorizer.adapt(x.values)
vectorized_text = vectorizer(x.values)

In [7]:
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(16)
dataset = dataset.prefetch(16)

In [8]:
train = dataset.take(int(len(dataset) * .7))
val = dataset.skip(int(len(dataset) * .7))
test = dataset.skip(int(len(dataset) * .9)).take(int(len(dataset) * .1))

In [9]:
train_generator = train.as_numpy_iterator()

# 2. Training

In [10]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding

In [11]:
model = Sequential([Embedding(MAX_FEATURES+1, 32),
                    Bidirectional(LSTM(32,activation='tanh')),
                    Dense(128,activation='relu'),
                    Dense(256,activation='relu'),
                    Dense(128,activation='relu'),
                    Dense(6,activation='sigmoid')])
model.compile(loss=tf.metrics.binary_crossentropy,optimizer='Adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          6400032   
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [1]:
history = model.fit(train, epochs=1, validation_data=val)

NameError: name 'model' is not defined

# 3. Make predictions

In [ ]:
input_text = vectorizer('fuck you!')

In [2]:
res = model.predict(np.expand_dims(input_text,0))

NameError: name 'model' is not defined